In [2]:
import pandas as pd

In [3]:
def create_time_windows(df, window_size, stride):
    # 입력 유효성 검사
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")
    if not isinstance(window_size, int) or window_size <= 0:
        raise ValueError("window_size must be a positive integer")
    if not isinstance(stride, int) or stride <= 0:
        raise ValueError("stride must be a positive integer")
    
    # 결과를 저장할 딕셔너리 생성
    result_dict = {}
    
    # 각 열에 대해 반복
    for column in df.columns:
        # 시간 윈도우 생성
        windows = {}
        for i in range(0, len(df) - window_size + 1, stride):
            window = df[column].iloc[i:i+window_size].values
            windows[df.index[i+window_size-1]] = window
        
        # 윈도우 데이터로 새 DataFrame 생성
        window_df = pd.DataFrame.from_dict(windows, orient='index')
        window_df.columns = [f"{column}_t-{window_size-i-1}" for i in range(window_size)]
        
        # 결과 딕셔너리에 추가
        result_dict[column] = window_df
    
    return result_dict

In [4]:
df = pd.read_parquet(r"D:\Workspace\DnS\data\AJ네트웍스_20190825_20240825.parquet")

In [5]:
df.shape

(1233, 6)

In [27]:
df.head(3).to_markdown()

'| 날짜                |   시가 |   고가 |   저가 |   종가 |   거래량 |    등락률 |\n|:--------------------|-------:|-------:|-------:|-------:|---------:|----------:|\n| 2019-08-26 00:00:00 |   4615 |   4615 |   4480 |   4540 |    34971 | -2.36559  |\n| 2019-08-27 00:00:00 |   4505 |   4585 |   4505 |   4560 |    20983 |  0.440529 |\n| 2019-08-28 00:00:00 |   4540 |   4640 |   4490 |   4580 |    20526 |  0.438596 |'

In [7]:
window_size = 5  # window 크기
stride = 2  # stride 크기
df_list = create_time_windows(df, window_size, stride)

In [8]:
from acf import calculate_acf
acf_df = calculate_acf(df['종가'], window_size=5)
acf_df

,lag_0,lag_1,lag_2,lag_3,lag_4
0,1.0,0.994347,0.988418,0.981891,0.97527
1,1.0,0.994347,0.988418,0.981891,0.97527
2,1.0,0.994347,0.988418,0.981891,0.97527
3,1.0,0.994347,0.988418,0.981891,0.97527
4,1.0,0.994347,0.988418,0.981891,0.97527
...,...,...,...,...,...
1228,1.0,0.994347,0.988418,0.981891,0.97527
1229,1.0,0.994347,0.988418,0.981891,0.97527
1230,1.0,0.994347,0.988418,0.981891,0.97527
1231,1.0,0.994347,0.988418,0.981891,0.97527


In [9]:
df['종가'].head

<bound method NDFrame.head of 날짜
2019-08-26    4540
2019-08-27    4560
2019-08-28    4580
2019-08-29    4650
2019-08-30    4700
              ... 
2024-08-19    4400
2024-08-20    4500
2024-08-21    4620
2024-08-22    4610
2024-08-23    4620
Name: 종가, Length: 1233, dtype: int64>

In [10]:
from buffett import calculate_buffett_index
buffett = calculate_buffett_index(df['종가'], 'KOR')
buffett.shape

(1233,)

In [11]:
from deMartini import demartini_index
de = demartini_index(df['종가'])
de

날짜
2019-08-26          NaN
2019-08-27          NaN
2019-08-28          NaN
2019-08-29          NaN
2019-08-30          NaN
                ...    
2024-08-19    49.585158
2024-08-20    55.881832
2024-08-21    62.013127
2024-08-22    61.249223
2024-08-23    61.756560
Name: rsi, Length: 1233, dtype: float64

In [12]:
from div_each_before import div_each_before

deb = div_each_before(df['종가'])
deb.shape

(1233,)

In [13]:
from fractional_difference import fractional_difference
fracdiff = fractional_difference(df['종가'], 0.3)
fracdiff.shape

(1233,)

In [14]:
from pivot import calculate_pivot_points
pivot_points = calculate_pivot_points(df['고가'], df['저가'], df['종가'])
pivot_points.shape

(1233, 7)

In [15]:

from sonar import sonar_indicator
sn = sonar_indicator(df, window_size=14)
sn.shape

(1233,)

In [16]:
from stocastic import stochastic_fast, stochastic_slow
stfa = stochastic_fast(df)
stsl = stochastic_slow(df)

In [17]:
stfa['fastd'].shape

(1233,)

In [18]:
stfa['fastk'].shape

(1233,)

In [19]:
from time_delay import time_delay_embedding
time_delay_df = time_delay_embedding(df['종가'], 60, 1)

In [20]:
time_delay_df['t-0']

2019-08-26    4540
2019-08-27    4560
2019-08-28    4580
2019-08-29    4650
2019-08-30    4700
              ... 
2024-08-19    4400
2024-08-20    4500
2024-08-21    4620
2024-08-22    4610
2024-08-23    4620
Name: t-0, Length: 1233, dtype: int64

In [21]:
from vix import calculate_vix
calVix = calculate_vix(df['종가'], window_size)
calVix.shape

(1233,)

In [22]:
from williams import williams_r
will = williams_r(df, 5) 
will.shape

(1233,)

In [23]:
import pandas as pd

def create_merged_indicators(df_dict):
    """
    Create a new DataFrame by merging multiple indicator DataFrames.
    
    :param df_dict: Dictionary of DataFrames or Series to merge
    :return: Merged DataFrame with all indicators
    """
    # Find the common index
    common_index = pd.Index([])
    for df in df_dict.values():
        if isinstance(df, pd.DataFrame) or isinstance(df, pd.Series):
            common_index = common_index.union(df.index)
    
    # Start with an empty DataFrame
    result = pd.DataFrame(index=common_index)
    
    for name, df in df_dict.items():
        # Ensure the index is aligned
        if isinstance(df, pd.DataFrame) or isinstance(df, pd.Series):
            df = df.reindex(common_index)
        
        # If df is a Series, convert to DataFrame
        if isinstance(df, pd.Series):
            df = df.to_frame(name=name)
        
        # If df has multiple columns, add a prefix
        if isinstance(df, pd.DataFrame) and len(df.columns) > 1:
            df = df.add_prefix(f"{name}_")
        
        # Merge
        result = pd.concat([result, df], axis=1)
    
    return result

# Example usage:
df_dict = {
    'ACF': acf_df,
    'Buffett': buffett,
    'DeMartini': de,
    'DivEachBefore': deb,
    'FracDiff': fracdiff,
    'PivotPoints': pivot_points,
    'Sonar': sn,
    'StochasticFast_K': stfa['fastk'],
    'StochasticFast_D': stfa['fastd'],
    'TimeDelay': time_delay_df['t-0'],
    'VIX': calVix,
    'Williams': will
}

# Create merged indicators DataFrame
merged_df = create_merged_indicators(df_dict)

print(merged_df.shape)

(2466, 22)


C:\Users\kimso\AppData\Local\Temp\ipykernel_29216\1081183072.py:14: RuntimeWarning: '<' not supported between instances of 'Timestamp' and 'int', sort order is undefined for incomparable objects.
  common_index = common_index.union(df.index)
C:\Users\kimso\AppData\Local\Temp\ipykernel_29216\1081183072.py:14: RuntimeWarning: '<' not supported between instances of 'Timestamp' and 'int', sort order is undefined for incomparable objects.
  common_index = common_index.union(df.index)
C:\Users\kimso\AppData\Local\Temp\ipykernel_29216\1081183072.py:14: RuntimeWarning: '<' not supported between instances of 'Timestamp' and 'int', sort order is undefined for incomparable objects.
  common_index = common_index.union(df.index)
C:\Users\kimso\AppData\Local\Temp\ipykernel_29216\1081183072.py:14: RuntimeWarning: '<' not supported between instances of 'Timestamp' and 'int', sort order is undefined for incomparable objects.
  common_index = common_index.union(df.index)
C:\Users\kimso\AppData\Local\Tem

In [24]:
acf_df

,lag_0,lag_1,lag_2,lag_3,lag_4
0,1.0,0.994347,0.988418,0.981891,0.97527
1,1.0,0.994347,0.988418,0.981891,0.97527
2,1.0,0.994347,0.988418,0.981891,0.97527
3,1.0,0.994347,0.988418,0.981891,0.97527
4,1.0,0.994347,0.988418,0.981891,0.97527
...,...,...,...,...,...
1228,1.0,0.994347,0.988418,0.981891,0.97527
1229,1.0,0.994347,0.988418,0.981891,0.97527
1230,1.0,0.994347,0.988418,0.981891,0.97527
1231,1.0,0.994347,0.988418,0.981891,0.97527
